# Notebook guide: linear viscoelastic ice cauldrons

Welcome!  This notebook will guide you through some of the key functions included in `cauldron_funcs.py` to simulate linear elastic/viscoelastic collapse of ice cauldrons.  Here we will use an idealized example.  

The subfolder `ESkafta-2015` applies these methods to the 2015 collapse of Eastern Skafta Cauldron, Vatnajokull, Iceland.  However, ArcticDEM and GPS data used to initialize and assess that example come from other sources and are not included in this repository.

### Import statements

First, we import the packages and modules we will use.  Remember to have your `jokull` environment (defined in `environment.yml` in this repository) activated to use these packages.

In [ ]:
import numpy as np
import scipy.misc as scp
from scipy import interpolate
from scipy.ndimage import gaussian_filter
# from osgeo import gdal
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle
import math
import sys
sys.path.insert(0, '/Users/lizz/Documents/GitHub/VE-cauldrons')
from cauldron_funcs import *

### Setting up cauldron geometry

If you have surface elevation data from a real cauldron, you can read it in as done in `ESkafta-2015/Skafta-ArcticDEM-transecting.py`.  Here we will initialize an idealized cauldron.

In [ ]:
transect_length = 2000 # m, diameter of the cauldron
npoints = 500
x_cylcoords = np.linspace(-0.5*transect_length, 0.5*transect_length, num=npoints) # x-axis values in cylindrical coordinates

In [ ]:
initial_elevation = 1200 # m, surface elevation of the cauldron before collapse
ice_thickness = 300 # m
cldrn = Cauldron(name='Example_Cauldron', 
                 initial_surface = lambda x: initial_elevation, # function relating initial elevation to x-coordinate
                 thickness = ice_thickness,
                 radius = 0.5*transect_length
                )

In [ ]:
cldrn.set_viscoelastic_bendingmod()

## Compute collapse profiles

Now we will use functions of our example `cldrn`, an instance of the `Cauldron` class, to compute post-collapse profiles of an elastic and linear viscoelastic plate.

### Elastic

In [ ]:
elastic_profile = [cldrn.LL_profile(x) for x in x_cylcoords] # LL for 'Landau & Lifshitz', analytical solution

In [ ]:
initial_surf = np.full(shape=npoints, fill_value=initial_elevation)

fig, ax = plt.subplots(1, figsize=(7, 3))
ax.plot(x_cylcoords, initial_surf, color='k', ls='-.', label='Pre-collapse surface')
ax.plot(x_cylcoords, elastic_profile, color='DarkBlue', lw=2, label='Elastic plate')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set(aspect=5,
       xlim=(-0.5*transect_length, 0.5*transect_length),
       ylim=(initial_elevation-100, initial_elevation+100),
       xlabel='Distance from cauldron center [m]',
       ylabel='Surface elevation [m a.s.l.]', 
       title='Elastic collapse with E={:.1E} Pa'.format(cldrn.youngmod))
ax.fill_between(x_cylcoords, initial_surf, elastic_profile, color='Gainsboro', hatch='/', edgecolor='DimGray', linewidth=0, alpha=0.7)
ax.fill_between(x_cylcoords, elastic_profile, (ax.get_ylim()[0]), color='Azure')
plt.show()

### Viscoelastic

Within `cauldron-funcs.py`, we apply a Laplace transform to take our analytical solution for the elastic case to the linear viscoelastic case.  This will take a little longer to run.  We will select only a few time slices to view, to minimize computational demand.

In [ ]:
days_of_collapse = 4 # how many days of collapse you want to simulate
nseconds = days_of_collapse*24*60*60
timestep = 40000 # seconds, approximately half a day
times = np.arange(0, nseconds, step=timestep)

VE_profiles = {t: [] for t in times}
for t in times:
    VE_profiles[t] = [cldrn.viscoelastic_profile(x, t) for x in x_cylcoords]

In [ ]:
colors = cm.get_cmap('winter_r')(np.linspace(0.1, 0.9, num=len(times)+1))
ylim_lower = float(np.min(VE_profiles[times[-1]])) - 100 # ensure display captures full collapse range

fig, ax1 = plt.subplots(1, figsize=(7, 3))
ax1.plot(x_cylcoords, initial_surf, color='k', ls='-.') #, label='15 Oct 2012'
ax1.plot(x_cylcoords, elastic_profile, color=colors[0], lw=2, label='Elastic plate')
for i,ti in enumerate(times[::2]):
    labeltime = int(round(ti/86400)) #time in days
    ax1.plot(x_cylcoords, VE_profiles[ti], ls='--', color=colors[i+1], lw=2, label='Viscoelastic, t = {} days'.format(labeltime))
ax1.legend(bbox_to_anchor=(1.05,1), loc='upper left') #
ax1.set(aspect=5,
        ylim=(ylim_lower, initial_elevation+100),
        xlim=(-0.5*transect_length, 0.5*transect_length),
        xlabel='Distance from cauldron center [m]',
        title='Viscoelastic collapse with E={:.1E} Pa, eta={:.1E} Pa s'.format(cldrn.youngmod, cldrn.dyn_viscos))
ax1.fill_between(x_cylcoords, y1=initial_surf, y2=[float(ve) for ve in VE_profiles[times[-1]]], color='Gainsboro', hatch='/', edgecolor='DimGray', linewidth=0, alpha=0.7)
ax1.fill_between(x_cylcoords, y1=[float(ve) for ve in VE_profiles[times[-1]]], y2=ylim_lower, color='Azure')
plt.show()

## Computing stress

We compute an approximation to the tensile/compressive stress at the surface of the ice.  Note that our approach here makes several assumptions that cause this approximation to be an _overestimate_: it will return a maximum plausible stress that could have been attained during collapse with this rheology.  See Ultee et al 2020, Journal of Glaciology for more details.

In [ ]:
elas_plate_stress = [cldrn.elastic_stress(x, config='radial_plate') for x in x_cylcoords]

In [ ]:
fig, ax2 = plt.subplots()
ax2.axhline(0, color='k', ls=':')
ax2.plot(x_cylcoords, 1e-6*np.asarray(elas_plate_stress))
ax2.fill_between(x_cylcoords, 1e-6*np.asarray(elas_plate_stress), 0, where=[sigma>0 for sigma in elas_plate_stress], color='DarkBlue', alpha=0.5, label='Compressive')
ax2.fill_between(x_cylcoords, 1e-6*np.asarray(elas_plate_stress), 0, where=[sigma<0 for sigma in elas_plate_stress], color='FireBrick', alpha=0.5, label='Tensile')
ax2.legend(bbox_to_anchor=(1.05,1), loc='upper left')
ax2.set(xlabel='Distance from cauldron center',
       ylabel='Max surface stress [MPa]',
       title='Elastic case')
plt.show()

In [ ]:
ve_plate_stress_init = [float(cldrn.viscoelastic_stress(x, times[0])) for x in x_cylcoords]
ve_plate_stress_final = [float(cldrn.viscoelastic_stress(x, times[-1])) for x in x_cylcoords]

In [ ]:
fig, ax3 = plt.subplots()
ax3.axhline(0, color='k', ls=':')
ax3.plot(x_cylcoords, 1e-6*np.asarray(ve_plate_stress_init))
ax3.fill_between(x_cylcoords, 1e-6*np.asarray(ve_plate_stress_init), 0, where=[sigma>0 for sigma in ve_plate_stress_init], color='DarkBlue', alpha=0.5, label='Compressive')
ax3.fill_between(x_cylcoords, 1e-6*np.asarray(ve_plate_stress_init), 0, where=[sigma<0 for sigma in ve_plate_stress_init], color='FireBrick', alpha=0.5, label='Tensile')
ax3.legend(bbox_to_anchor=(1.05,1), loc='upper left')
ax3.set(xlabel='Distance from cauldron center',
       ylabel='Max surface stress [MPa]',
       title='Viscoelastic case, E={:.1E}, eta={:.1E}'.format(cldrn.youngmod, cldrn.dyn_viscos))
plt.show()

Note that in this simple linear viscoelastic case, there is no creep, crevassing, etc. that would concentrate or dissipate stress.  Unphysically high values of surface stress are possible toward the end of the collapse period, when such effects would be expected to substantially modify the stress in nature.

## Modifying material properties

The `Cauldron` instance we set up above has inherited default material properties from the `Ice` class defined in `cauldron_funcs.py`.  

You can examine the material properties (density `rho_ice`, Young's modulus `youngmod`, Poisson's ratio `poisson_nu`, and dynamic viscosity `dyn_viscos`) like this:

In [ ]:
cldrn.youngmod

You can also change the material constants by calling them like this:

In [ ]:
cldrn.youngmod = 2E9 # Pa

We can now re-run the analysis cells with an altered Young's modulus (or other material property) to explore the parameter space.